In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICE'] = ''
import random

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from omegaconf import OmegaConf
import torch
%load_ext autoreload
%autoreload 2

from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')

In [2]:
from hydra import compose, initialize
from omegaconf import OmegaConf
from hydra.utils import instantiate

In [15]:
path = "../training/conf/network/mammut_lines.yaml"

In [45]:
with open(path, "rb") as f:
    cfg = OmegaConf.load(f)

In [46]:
cfg

{'_target_': 'text_recognizer.network.mammut.MaMMUT', 'encoder': {'_target_': 'text_recognizer.network.vit.Vit', 'image_height': 56, 'image_width': 1024, 'patch_height': 56, 'patch_width': 8, 'dim': 512, 'encoder': {'_target_': 'text_recognizer.network.transformer.encoder.Encoder', 'dim': 512, 'heads': 12, 'dim_head': 64, 'ff_mult': 4, 'depth': 4, 'dropout_rate': 0.1}, 'channels': 1}, 'image_attn_pool': {'_target_': 'text_recognizer.network.transformer.attention.Attention', 'dim': 512, 'heads': 8, 'causal': False, 'dim_head': 64, 'ff_mult': 4, 'dropout_rate': 0.0, 'use_flash': True, 'norm_context': True, 'rotary_emb': None}, 'decoder': {'_target_': 'text_recognizer.network.transformer.decoder.Decoder', 'dim': 512, 'ff_mult': 4, 'heads': 12, 'dim_head': 64, 'depth': 6, 'dropout_rate': 0.1}, 'dim': 512, 'dim_latent': 512, 'num_tokens': 58, 'pad_index': 3, 'num_image_queries': 256}

In [47]:
net = instantiate(cfg)

In [39]:
from torchinfo import summary

In [50]:
summary(net, ((1, 1, 56, 1024), (1, 89)), device="cpu", depth=4)

Layer (type:depth-idx)                             Output Shape              Param #
MaMMUT                                             [1, 89, 58]               627,712
├─Vit: 1-1                                         [1, 128, 512]             --
│    └─Sequential: 2-1                             [1, 128, 512]             --
│    │    └─Rearrange: 3-1                         [1, 128, 448]             --
│    │    └─LayerNorm: 3-2                         [1, 128, 448]             896
│    │    └─Linear: 3-3                            [1, 128, 512]             229,888
│    │    └─LayerNorm: 3-4                         [1, 128, 512]             1,024
│    └─Encoder: 2-2                                [1, 128, 512]             --
│    │    └─ModuleList: 3-5                        --                        --
│    │    │    └─Attention: 4-1                    [1, 128, 512]             4,724,224
│    │    │    └─Attention: 4-2                    [1, 128, 512]             4,724,224
│    │ 

In [48]:
t = net(torch.randn(1, 1, 56, 1024), torch.randint(1, 4, (1, 4)))

In [49]:
t.shape

torch.Size([1, 4, 58])